# 連結雲端磁碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/109_台大經研碩士在職/110學年度/碩二上學期/資料科學與社會研究/Project/台中市交通事故/各縣市資料'
!ls

# 引入套件

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime, date, time

# 台中

## 臺中市政府OAS標準之API說明文件:
https://datacenter.taichung.gov.tw/swagger/yaml/387130000C 

In [ ]:
url = "https://datacenter.taichung.gov.tw/swagger/yaml/387130000C"
swagger = requests.get(url).json()

In [ ]:
swagger["paths"]['/OpenData/8fd91c29-6e97-4a8a-8248-06ddff5fbaa0']['get']['tags'][0]
#  想辦法找出 '/OpenData/8fd91c29-6e97-4a8a-8248-06ddff5fbaa0'

'2019_07_10959-03-01-2_臺中市執行家戶訪查工作成果統計'

### 擷取 10801 迄今之資料

In [ ]:
def getapi(key_tag='月份交通事故資料-CSV',year=['108','109','110'], swagger=swagger):
    """Get api from Taichung gov swagger

    Input key word string, then get the apis

    Args:
        key_tag (str): target document name. default '月份交通事故資料-CSV'
        year (list): list of string. default ['108','109','110']
        swagger (dict): default is swagger of Taichung gov

    Returns:
        apis (dict): {aip: tag}
    """
    apis = {}
    for api in swagger["paths"].keys():
        tag = swagger["paths"][api]['get']['tags'][0]
        if key_tag in tag:
            for y in year:
                if y in tag:
                    apis[swagger["paths"][api]['get']['tags'][0]] = api
                    break
    return apis
apis = getapi()

### 合併為 DataFrame

In [ ]:
urlroot = 'https://datacenter.taichung.gov.tw/swagger'
col_dict = {tag:pd.read_csv(urlroot+api).columns for tag, api in apis.items()}
col_dict.keys()

dict_keys(['臺中市政府警察局108年10月份交通事故資料-CSV', '臺中市政府警察局108年11月份交通事故資料-CSV', '臺中市政府警察局108年12月份交通事故資料-CSV', '臺中市政府警察局108年1月份交通事故資料-CSV', '臺中市政府警察局108年2月份交通事故資料-CSV', '臺中市政府警察局108年3月份交通事故資料-CSV', '臺中市政府警察局108年4月份交通事故資料-CSV', '臺中市政府警察局108年5月份交通事故資料-CSV', '臺中市政府警察局108年6月份交通事故資料-CSV', '臺中市政府警察局108年7月份交通事故資料-CSV', '臺中市政府警察局108年8月份交通事故資料-CSV', '臺中市政府警察局108年9月份交通事故資料-CSV', '臺中市政府警察局109年10月份交通事故資料-CSV', '臺中市政府警察局109年11月份交通事故資料-CSV', '臺中市政府警察局109年12月份交通事故資料-CSV', '臺中市政府警察局109年1月份交通事故資料-CSV', '臺中市政府警察局109年2月份交通事故資料-CSV', '臺中市政府警察局109年3月份交通事故資料-CSV', '臺中市政府警察局109年4月份交通事故資料-CSV', '臺中市政府警察局109年5月份交通事故資料-CSV', '臺中市政府警察局109年6月份交通事故資料-CSV', '臺中市政府警察局109年7月份交通事故資料-CSV', '臺中市政府警察局109年8月份交通事故資料-CSV', '臺中市政府警察局109年9月份交通事故資料-CSV', '臺中市政府警察局110年1月份交通事故資料-CSV', '臺中市政府警察局110年2月份交通事故資料-CSV', '臺中市政府警察局110年3月份交通事故資料-CSV', '臺中市政府警察局110年4月份交通事故資料-CSV', '臺中市政府警察局110年5月份交通事故資料-CSV', '臺中市政府警察局110年6月份交通事故資料-CSV', '臺中市政府警察局110年7月份交通事故資料-CSV'])

In [ ]:
urlroot = 'https://datacenter.taichung.gov.tw/swagger/'
listdf = [pd.read_csv(urlroot+f) for f in apis.values()]

In [ ]:
newlist = []
for df in listdf:
    if df.shape[1] != 49:
        df.columns = list(col_dict['臺中市政府警察局110年7月份交通事故資料-CSV'])
        df = df.drop(['2-30'], axis=1)
        df.columns = list(col_dict['臺中市政府警察局108年3月份交通事故資料-CSV'])
    # print(df.shape[1])
    newlist.append(df)

In [ ]:
traffic = pd.concat(newlist)
traffic = traffic.rename(columns={'年': 'year', '月': 'month', '日': 'day', 
                                  '時': 'hour', '分':'minute'})
traffic['datetime'] = pd.to_datetime(traffic[['year', 'month', 'day', 'hour', 'minute']])
traffic.sort_values(['datetime']).reset_index(inplace=True)

In [ ]:
traffic.head(2)

,year,month,day,hour,minute,縣市,區,死,受傷,天候,光線,道路類別,速限,道路型態,事故位置,路面鋪裝,路面狀態,路面缺陷,障礙物,視距,號誌種類,號誌動作,分向設施,快車道或一般車道間,快慢車道間,路面邊線,事故類型及型態,主要肇因,受傷程度,主要傷處,保護裝備,行動電話,當事者區分,車輛用途,當事者行動狀態,駕駛資格情形,駕駛執照種類,飲酒情形,車輛撞擊部位最初,車輛撞擊部位其他,肇事因素個別,肇事因素主要,肇事逃逸,職業,旅次目的,車種,GPS座標緯度,GPS座標經度,事故類別,datetime
0,2019.0,10.0,1.0,0.0,14.0,臺中市,大里區,0.0,2.0,8.0,1.0,5.0,50,4.0,1.0,1.0,4.0,4.0,5.0,7.0,3.0,1.0,10.0,5.0,5.0,2.0,15.0,6.0,3,10,1,1,B03,7,9,1,8,2,5,NaN,6,6,1,21,9,B03,24.096579,120.698988,2.0,2019-10-01 00:14:00
1,2019.0,10.0,1.0,2.0,30.0,臺中市,西屯區,0.0,0.0,8.0,NaN,NaN,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B12,NaN,NaN,1,NaN,2,NaN,NaN,23,NaN,1,22,NaN,B12,24.159105,120.639130,3.0,2019-10-01 02:30:00


### 存檔

In [ ]:
traffic.to_csv('taichung.csv', index=False)

In [ ]:
traffic.groupby(pd.Grouper(key='datetime',freq='M')).count()

### 資料檢視
1. 2-30 與 2_30
2. 當事者區分
3. GPS經緯度
4. NaN

In [ ]:
traffic.shape

(212869, 50)

In [ ]:
traffic.isna().sum(axis=1)

0        7
1       34
2        7
3       32
4        7
        ..
6450     7
6451     7
6452     7
6453     7
6454    33
Length: 212869, dtype: int64

# 桃園

## 桃園市交通事故資料表-含第二當事人
1. 提供桃園市交通事故資料彙整，資料包含事故現場環境及相關肇事因素等。
2. 資料包含所有當事人，可使用當事人順位欄位進行篩選。
3. 交通事故資料表自106年起更新格式，整合經緯度及代碼對照。
4. 107年起新增當事人性別及年齡欄位。

In [2]:
tycg_109df = pd.read_csv('https://quality.data.gov.tw/dq_download_csv.php?nid=129320&md5_url=99beeb75a05e72b615e4062d0c7d1546')
tycg_110df = pd.read_csv('https://quality.data.gov.tw/dq_download_csv.php?nid=143048&md5_url=490676adda932a70c763d9b5cd214b2e')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,20,22,28,30,37,39,41,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,96,98,100,102,104,106,108,110,112,114,116,118,120,122,128,130,132,134) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# tycg_109df.columns == tycg_110df.columns
# tycg_109df.groupby(by=['發生年度', '發生月份']).count()['發生日期']
# tycg_110df.groupby(by=['發生年度', '發生月份']).count()['發生日期']
tycg_109df = tycg_109df[tycg_109df['發生年度'] == 2020] 

In [4]:
tycg = pd.concat([tycg_109df, tycg_110df])

In [7]:
print(list(tycg.columns))

['發生日期', '發生時間', '發生年度', '發生月份', '發生星期', '到場處理日期', '到場處理時間', '結束_事故排除_日期', '結束_事故排除_時間', 'GPS經度', 'GPS緯度', '事故類別名稱', '地址類型名稱', '發生縣市名稱', '發生市區鄉鎮名稱', '發生地址_村里代碼', '發生地址_村里名稱', '發生地址_鄰', '發生地址_路街代碼', '發生地址_路街', '發生地址_段', '發生地址_巷', '發生地址_弄', '發生地址_號', '發生地址_前幾公尺', '發生地址_側名稱', '發生交叉路口_村里名稱', '發生交叉路口_路街口', '發生交叉路口_段', '發生交叉路口_巷', '發生交叉路口_弄', '發生地址_其他', '死亡人數_24小時內', '死亡人數_2_30日內', '受傷人數', '天候代碼', '天候名稱', '光線代碼', '光線名稱', '道路第1當事者_代碼', '道路第1當事者_名稱', '速限_第1當事者', '道路型態大類別代碼', '道路型態大類別名稱', '道路型態子類別代碼', '道路型態子類別名稱', '事故位置大類別代碼', '事故位置大類別名稱', '事故位置子類別代碼', '事故位置子類別名稱', '路面狀況_路面鋪裝代碼', '路面狀況_路面鋪裝名稱', '路面狀況_路面狀態代碼', '路面狀況_路面狀態名稱', '路面狀況_路面缺陷代碼', '路面狀況_路面缺陷名稱', '道路障礙_障礙物代碼', '道路障礙_障礙物名稱', '道路障礙_視距品質代碼', '道路障礙_視距品質名稱', '道路障礙_視距代碼', '道路障礙_視距名稱', '號誌_號誌種類代碼', '號誌_號誌種類名稱', '號誌_號誌動作代碼', '號誌_號誌動作名稱', '車道劃分設施_分向設施大類別代碼', '車道劃分設施_分向設施大類別名稱', '車道劃分設施_分向設施子類別代碼', '車道劃分設施_分向設施子類別名稱', '車道劃分設施_快車道或一般車道間代碼', '車道劃分設施_快車道或一般車道間名稱', '車道劃分設施_快慢車道間代碼', '車道劃分設施_快慢車道間名稱', '車道劃分設施_路面邊線代碼', '車道劃分設施_路面邊線名稱', '事故類型及型態大類別代碼', '

## 處理時間

In [ ]:
def time_col(number):
    try:
        number = int(number)
        h = number // 10000
        m = (number // 100) - (h * 100)
        s = number % 100
        t = time(h,m,s)
    except:
        t = np.nan
    return t

tycg['occur_date'] = pd.to_datetime(tycg['發生日期'], format='%Y%m%d')
tycg['occur_time'] =  pd.to_datetime(tycg['發生時間'].apply(time_col), format='%H:%M:%S')
tycg['occur_datetime'] = pd.to_datetime(tycg['occur_date'].dt.date.astype(str) + ' ' + tycg['occur_time'].dt.time.astype(str))

tycg['start_date'] = pd.to_datetime(tycg['到場處理日期'], format='%Y%m%d')
tycg['start_time'] =  pd.to_datetime(tycg['到場處理時間'].apply(time_col), format='%H:%M:%S')
tycg['start_datetime'] = pd.to_datetime(tycg['start_date'].dt.date.astype(str) + ' ' + tycg['start_time'].dt.time.astype(str), errors='coerce')

tycg['end_date'] = pd.to_datetime(tycg['結束_事故排除_日期'], format='%Y%m%d')
tycg['end_time'] =  pd.to_datetime(tycg['結束_事故排除_時間'].apply(time_col), format='%H:%M:%S')
tycg['end_datetime'] = pd.to_datetime(tycg['end_date'].dt.date.astype(str) + ' ' + tycg['end_time'].dt.time.astype(str), errors='coerce')

## 存檔

In [ ]:
tycg.to_csv('taoyuan.csv', index=False)